# Post-processing Spectral Operations

In a post-processing task, computing derivatives is a common operation. Rayleigh computes derivatives using spectral transforms and recursion relations. This notebook describes a python
module that allows the computation of various spectral transforms and derivatives (*spectral_utils.py*).

This notebook assumes that you are familiar with running Rayleigh and comfortable with manipulating the various types of diagnostic outputs using the provided python tools.

Contents:
+        Generate the data for this tutorial
+       Extracting the data
+      Using the 4 main python classes
 1.       Fourier
 2.       Legendre
 3.       Chebyshev
 4.       SHT
+       Important notes and restrictions

================================================================================================


# I. Generate the Data

To fully understand this notebook, a particular model setup with specific outputs must be generated. Start by copying the *c2001_case0_minimal* input example (Boussinesq hydro benchmark) into a working directory and then rename it *main_input*.

There are three main changes that need to be made:

 1. The radial grid
 2. The output quantities
 3. The number of iterations to use


## Change the radial grid
The first modification will be in the problemsize namelist. In the *main_input* file change the *problemsize_namelist* to be:
```
&problemsize_namelist
 l_max = 63
 domain_bounds = 0.53846153846153832d0, 1.04d0, 1.34d0, 1.5384615384615383d0
 ncheby = 26,20,18
 dealias_by = 2,1,0
/
```
This will build a radial domain composed of three sub-domains, each with a different resolution. The boundaries of the sub domains are explicitly set and the resulting domain should have an aspect ratio of 0.35 and a shell depth of 1.0, consistent with the chosen benchmark mode.

These choices represent the most general radial grid setup in order to showcase the python interface. Examples of radial grid setups that are more commonly used will be discussed below, but not explicitly executed.

## Add some output quantities
The next modification will be to add specific output quantities. In the same *main_input* file change the *output_namelist* to be:
```
&output_namelist
 shellavg_values = 1,2,3,10,19,28,11,20,29,12,21,30,501,507,513,519
 shellavg_frequency = 10
 shellavg_nrec = 10

 shellslice_values = 1,2,3,10,19,28,11,20,29,12,21,30,501,507,513,519
 shellslice_frequency = 10
 shellslice_nrec = 10
 shellslice_levels_nrm = 0.5, 0.67, 0.8, 0.9

 shellspectra_values = 1,2,3,10,19,28,11,20,29,12,21,30,501,507,513,519
 shellspectra_frequency = 10
 shellspectra_nrec = 10
 shellspectra_levels_nrm = 0.5, 0.67, 0.8, 0.9

 azavg_values = 1,2,3,10,19,28,11,20,29,12,21,30,501,507,513,519
 azavg_frequency = 10
 azavg_nrec = 10
/
```
The added quantities correspond to:

| Quantity Code | Description |
|-------------- | ------------|
| 1             | $v_r$ |
| 2             | $v_\theta$ |
| 3             | $v_\phi$ |
| 10            | $\frac{\partial v_r}{\partial r}$ |
| 19            | $\frac{\partial v_r}{\partial \theta}$ |
| 28            | $\frac{\partial v_r}{\partial \phi}$ |
| 11            | $\frac{\partial v_\theta}{\partial r}$ |
| 20            | $\frac{\partial v_\theta}{\partial \theta}$ |
| 29            | $\frac{\partial v_\theta}{\partial \phi}$ |
| 12            | $\frac{\partial v_\phi}{\partial r}$ |
| 21            | $\frac{\partial v_\phi}{\partial \theta}$ |
| 30            | $\frac{\partial v_\phi}{\partial \phi}$ |
| 501           | $\Theta$ |
| 507           | $\frac{\partial\Theta}{\partial r}$ |
| 513           | $\frac{\partial\Theta}{\partial \theta}$ |
| 519           | $\frac{\partial\Theta}{\partial \phi}$ |

The last change is to adjust the *max_iterations*. The final *main_input* file should look like this:
```
&problemsize_namelist
 l_max = 63
 domain_bounds = 0.53846153846153832d0, 1.04d0, 1.34d0, 1.5384615384615383d0
 ncheby = 26,20,18
 dealias_by = 2,1,0
/
&numerical_controls_namelist
/
&physical_controls_namelist
 benchmark_mode = 1
 benchmark_integration_interval = 100
 benchmark_report_interval = 500
/
&temporal_controls_namelist
 max_iterations = 300
 checkpoint_interval = 100000
 quicksave_interval = 10000
 num_quicksaves = 2
/
&io_controls_namelist
/
&output_namelist
 shellavg_values = 1,2,3,10,19,28,11,20,29,12,21,30,501,507,513,519
 shellavg_frequency = 10
 shellavg_nrec = 50

 shellslice_values = 1,2,3,10,19,28,11,20,29,12,21,30,501,507,513,519
 shellslice_frequency = 10
 shellslice_nrec = 50
 shellslice_levels_nrm = 0.5, 0.67, 0.8, 0.9

 shellspectra_values = 1,2,3,10,19,28,11,20,29,12,21,30,501,507,513,519
 shellspectra_frequency = 10
 shellspectra_nrec = 50
 shellspectra_levels_nrm = 0.5, 0.67, 0.8, 0.9

 azavg_values = 1,2,3,10,19,28,11,20,29,12,21,30,501,507,513,519
 azavg_frequency = 10
 azavg_nrec = 50
/

&Boundary_Conditions_Namelist
/
&Initial_Conditions_Namelist
/
&Test_Namelist
/
&Reference_Namelist
/
&Transport_Namelist
/
```

## Run Rayleigh to generate the code
Now that the *main_input* file has been properly configured, it is time to run Rayleigh and generate some data. The purpose of this notebook is to showcase the python interface of the *spectral_utils.py* module, the benchmark does not need to run to completion.

If the *max_iterations* value is changed, be sure to change the *_frequency* and *_nrec* variables in each output quantity accordingly.

Using the *main_input* file as configured above (benchmark mode turned on), the code will run to completion in about 45 seconds using only 16 cores (about 6-7 iter/sec).

# II. Extracting the Data
After generating the data, it must be read into python. You must ensure that the *rayleigh_diagnostics.py* file can be imported into python: either adjust your PYTHONPATH or copy the *rayleigh_diagnostics.py* file into your working directory.

The *spectral_utils.py* file will also be needed (it lives in the same directory as *rayleigh_diagnostics*.py), set your PYTHONPATH or copy this file to the working directory as well.

We will be reading AZ_Avgs, Shell_Avgs, Shell_Spectra, and Shell_Slices data.

In [ ]:
%matplotlib inline
from rayleigh_diagnostics import Shell_Avgs, AZ_Avgs, Shell_Slices, Shell_Spectra
import spectral_utils as SU
import matplotlib.pyplot as plt
import numpy as np
import os

Define a helper routine to report errors between two arrays

In [ ]:
def report_error(x, y, title):
    error = np.abs(x - y)
    #print("{} maximum error = {}".format(title, np.max(error)))
    #print("{} average error = {}".format(title, np.mean(error)))
    print("{} median error = {}".format(title, np.median(error)))

If modifications were made to *max_iterations*, the *_frequency* values, or the *_nrec* values, then the following paths to the output files will need to be changed for your particular case. Be sure the *data_dir* value is changed accordingly.

In [ ]:
data_dir = "/home/orvedahl/cueball/Rayleigh-Runs/Testing/Spectral-Operations/"

shavg_itr  = "00000500"
azavg_itr  = "00000500"
shslc_itr  = "00000500"
shspec_itr = "00000500"

shavg  = Shell_Avgs(   os.path.join(data_dir, "Shell_Avgs",    shavg_itr),  path='')
azavg  = AZ_Avgs(      os.path.join(data_dir, "AZ_Avgs",       azavg_itr),  path='')
shslc  = Shell_Slices( os.path.join(data_dir, "Shell_Slices",  shslc_itr),  path='')
shspec = Shell_Spectra(os.path.join(data_dir, "Shell_Spectra", shspec_itr), path='')

In [ ]:
inds     = [1,  2, 3,501] # useful values for interfacing with the lookup table
dr_inds  = [10,11,12,507]
dth_inds = [19,20,21,513]
dp_inds  = [28,29,30,519]

nth = shslc.ntheta
nphi = shslc.nphi

radius = azavg.radius             # extract/build the Rayleigh grids
theta = np.arccos(shslc.costheta)
dphi = 2*np.pi/nphi
phi = dphi*np.arange(nphi)

# Shell Slice data
q_slc    = shslc.vals[:,:,:,shslc.lut[    inds],:] # shape (nphi,nth,nradii,3,ntime)
dqdt_slc = shslc.vals[:,:,:,shslc.lut[dth_inds],:]
dqdp_slc = shslc.vals[:,:,:,shslc.lut[ dp_inds],:]

# Shell Avgs data
q_shavg    = shavg.vals[:,0,shavg.lut[    inds],:] # shape (nr,3,nt)
dqdr_shavg = shavg.vals[:,0,shavg.lut[ dr_inds],:]

# Az Avgs data
q_az    = azavg.vals[:,:,azavg.lut[    inds],:] # shape (nth,nr,3,nt)
dqdr_az = azavg.vals[:,:,azavg.lut[ dr_inds],:]
dqdt_az = azavg.vals[:,:,azavg.lut[dth_inds],:]

# Shell Spectra data
q_sp    = shspec.vals[:,:,:,shspec.lut[    inds],:] # shape (l,m,nr,3,nt)
dqdt_sp = shspec.vals[:,:,:,shspec.lut[dth_inds],:]
dqdp_sp = shspec.vals[:,:,:,shspec.lut[ dp_inds],:]

# III. Using the Spectral Objects
The *spectral_utils.py* file contains 4 classes:

1. *Fourier*
2. *Legendre*
3. *Chebyshev*
4. *SHT*

Each will be explored in turn.

## 1) Fourier
The *Fourier* class is designed to compute Fourier transforms of Rayleigh data. It also provides a method to compute derivatives with respect to phi/longitude.

It is initialized by supplying the resolution in the phi direction

In [ ]:
F = SU.Fourier(nphi)

Use the *Fourier* object to compute the derivative. The *d_dphi* method can compute the derivative of data that is already in spectral space: the *physical=True* argument specifies that the data is in physical space (the default).

In [ ]:
phi_deriv = F.d_dphi(q_slc, axis=0, physical=True)

report_error(phi_deriv, dqdp_slc, "d/dphi")

Compare this error to a 6th order finite difference method, available in the *spectral_utils.py* module.

In [ ]:
dqdx = SU.ddx(q_slc, phi, axis=0)

report_error(dqdp_slc, dqdx, "6th FD")

The *Fourier* class can also compute forward and inverse Fourier transforms

In [ ]:
hybrid = F.to_spectral(q_slc, axis=0) # output will have shape (nfreq,nth,nradii,3,ntime)

phys = F.to_physical(hybrid, axis=0)

report_error(q_slc, phys, "")

## 2) Legendre
The *Legendre* class is designed to compute Legendre transforms of Rayleigh data. It also provides a method to compute derivatives with respect to theta/co-latitude.

It is initialized by supplying the resolution in the theta direction

In [ ]:
L = SU.Legendre(nth)

Use the *Legendre* object to compute the derivative. The *d_dtheta* method can compute the derivative of data that is already in spectral space: the *physical=True* argument specifies that the data is in physical space (the default).

In [ ]:
th_deriv = L.d_dtheta(q_slc, axis=1)

report_error(th_deriv, dqdt_slc, "d/dth")

Compare this to the 6th order finite difference method, which supports nonuniform grids

In [ ]:
dqdx = SU.ddx(q_slc, theta, axis=1)

report_error(dqdt_slc, dqdx, "6th FD")

The *Legendre* class can also compute forward and inverse Legendre transforms

In [ ]:
hybrid = L.to_spectral(q_slc, axis=1)

phys = L.to_physical(hybrid, axis=1)

report_error(q_slc, phys, "")

The derivative can also be computed from spectral space, just be sure to tell the derivative routine that the data is in spectral space.

In [ ]:
hybrid = L.to_spectral(q_az, axis=0) # transform the AZ_Avgs data

th_deriv_hybrid = L.d_dtheta(hybrid, axis=0, physical=False)

th_deriv = L.to_physical(th_deriv_hybrid, axis=0) # transform back to physical space to compute error

report_error(th_deriv, dqdt_az, "d/dth")

# Quick Note on Expected Accuracy
All of the classes within the *spectral_utils.py* module rely on expanding the input data into a series of basis functions. There are some functions that do not behave well when written as a truncated expansion. Typically, this class of function requires power in all modes of the expansion, up to and including the maximum degree in the truncation. This fact usually leads to underresolved features and a poor final result.

As an example, below is a small scaling test to compute the derivative of two simple functions using the *Legendre* class. The two functions are $F_1 = \frac{1}{\cos\theta}$ and $F_2 = \sin\theta$. The median error on the grid is reported along with the 6th order finite difference error.

In [ ]:
print("\n\t\t\tMedian absolute error\n")
print("Nth\tL.d_dtheta F1\tL.d_dtheta F2\t6th FD F1\t6th FD F2")
print("-"*70)
for n in [16,32,64,128,256,512]:
    Ltmp = SU.Legendre(n)
    th = Ltmp.theta

    # first function = 1/cos(th)
    F1 = 1./np.cos(th)
    dF1dth_true = np.sin(th)/np.cos(th)**2
    dF1dth = Ltmp.d_dtheta(F1)
    dF1dthFD = SU.ddx(F1, th)

    err1 = np.median(np.abs(dF1dth - dF1dth_true))
    err1FD = np.median(np.abs(dF1dthFD - dF1dth_true))
    
    # second function = sin(th)
    F2 = np.sin(th)
    dF2dth_true = np.cos(th)
    dF2dth = Ltmp.d_dtheta(F2)
    dF2dthFD = SU.ddx(F2, th)
    
    err2 = np.median(np.abs(dF2dth - dF2dth_true))
    err2FD = np.median(np.abs(dF2dthFD - dF2dth_true))
    
    # report the errors
    print("{}\t{:.6f}\t{:.6f}\t{:.6e}\t{:.6e}".format(n, err1, err2, err1FD, err2FD))

## 3) Chebyshev
The *Chebyshev* class is designed to compute Chebyshev transforms of Rayleigh data. It also provides a method to compute derivatives with respect to radius.

The most basic radial grid is defined by global domain bounds and a single resolution. For example, a domain using 72 grid points with a lower bound of 0.5 and an upper bound of 2.5, can be built in one of three ways:
```
    C_1 = SU.Chebyshev(72, rmin=0.5, rmax=2.5)

    C_2 = SU.Chebyshev(72, aspect_ratio=0.2, shell_depth=2)

    C_3 = SU.Chebyshev(72, boundaries=(0.5,2.5))
```
All three options will produce identical grids.

To build the above domain using 3 uniformly spaced subdomains:
```
    C_uniform = SU.Chebyshev(24, rmin=0.5, rmax=2.5, n_uniform_domains=3)
```
In the above example, the resolution refers to the resolution within each subdomain, giving the total grid 72 points, just as before.

The most general way to build the radial grid is to define the internal boundaries, the resolution within each subdomain, and if desired, change the amount of dealiasing. This was done in our input file at the start of this tutorial. In this case, the resolution is specified as a tuple or list of the resolution within each subdomain. The boundaries are supplied as a tuple or list with N+1 entries, given N resolutions. The first and last elements of the boundaries entry provide the global domain bounds. To build the *Chebyshev* object, prior knowledge of the radial grid is required. For our input example defined above:

In [ ]:
bounds = (0.53846153846153832, 1.04, 1.34, 1.5384615384615383)
C = SU.Chebyshev((26,20,18), dealias=(2,1,0), boundaries=bounds)

Use the *Chebyshev* object to compute the derivative. The *d_dr* method can compute the derivative of data that is already in spectral space: the *physical=True* argument specifies that the data is in physical space (the default).

In [ ]:
r_deriv = C.d_dr(q_az, axis=1)

report_error(r_deriv, dqdr_az, "d/dr")

Compare this to the 6th order finite difference method, which allows nonuniform grids. The radial grid in Rayleigh involves duplicate values when subdomains are used. This means a modified 6th order finite difference method must be used, which requires an extra argument that provides the indices of the repeated grid points. The index information is computed and stored in the *Chebyshev* object.

In [ ]:
dqdx = SU.ddx_repeated_gridpoints(q_az, radius, C.boundary_indices, axis=1)

report_error(dqdr_az, dqdx, "6th FD")

The *Chebyshev* class can also compute forward and inverse Chebyshev transforms

In [ ]:
hybrid = C.to_spectral(q_shavg, axis=0)

phys = C.to_physical(hybrid, axis=0)

report_error(q_shavg, phys, "")

The derivative routine accepts incoming data in spectral space, just use the *physical=False* keyword.

In [ ]:
r_deriv = C.d_dr(hybrid, axis=0, physical=False)

r_deriv = C.to_physical(r_deriv, axis=0) # transform back to physical to compute error

report_error(r_deriv, dqdr_shavg, "d/dr")

## 4) SHT
The *SHT* class will perform full spherical harmonic transforms. It is initialized by specifying the resolution in the theta direction. There are two ways to do this: use the physical space resolution or the spectral space resolution.
```
    S_1 = SU.SHT(n_theta)
    
    S_2 = SU.SHT(l_max, spectral=True)
```
The default behavior is to treat the first argument as the physical space resolution.

In [ ]:
S = SU.SHT(nth)

The *SHT* object offers a full SHT transform to spectral space

In [ ]:
spec = S.to_spectral(q_slc, th_l_axis=1, phi_m_axis=0) # output will be (nm,nl,nr,3,nt)
spec = np.swapaxes(spec, 0, 1)                         # but Spectra are (nl,nm,nr,3,nt)

report_error(q_sp, spec, "SHT")

and a full SHT transfrom to physical space

In [ ]:
phys = S.to_physical(q_sp, th_l_axis=0, phi_m_axis=1) # output will be (nth,nphi,nr,3,nt)
phys = np.swapaxes(phys, 0, 1)                        # but Slices are (nphi,nth,nr,3,nt)

report_error(q_slc, phys, "iSHT")

There is also the ability to compute derivatives with respect to phi, but only from within spectral space

In [ ]:
dp_deriv = S.d_dphi(q_sp, m_axis=1)

report_error(dqdp_sp, dp_deriv, "SHT d/dphi")

Derivatives with respect to theta are computed as $\sin\theta\frac{\partial F}{\partial\theta}$ from within spectral space.

In [ ]:
shp = [1]*len(dqdt_slc.shape) # build sin(th)*dq/dth from Shell Slices
shp[1] = -1
sinth = np.reshape(shslc.sintheta, tuple(shp)) # sinth is now same shape as Shell Slice data
s_dqdt_slc = sinth*dqdt_slc

s_dqdt_sp = S.to_spectral(s_dqdt_slc, th_l_axis=1, phi_m_axis=0) # transform to spectral
s_dqdt_sp = np.swapaxes(s_dqdt_sp, 0, 1)                         # and ensure shapes are same

# Note: the above computation is only needed to compute the error below

s_deriv = S.sin_d_dtheta(q_sp, l_axis=0, m_axis=1) # compute sin(th)*dq/dth in spectral space


report_error(s_dqdt_sp, s_deriv, "SHT sin(th) d/dth")